In [9]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np

import xarray as xr

import regionmask

import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import cartopy.feature as cfeat
import matplotlib.patches as mpatches
import datetime

import xesmf as xe

import geopandas
from shapely.geometry import Point, Polygon

import feather

import glob

import calendar

import sys
sys.path.append('../')
import utils
import plotting

np.seterr(invalid='ignore'); # disable a warning from matplotlib and cartopy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
##### import files and change hydro #####

###get ORIS, plant type, and Region Name from our modified generation file
oris_nonuc_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_no-nuclear_all-generators_20k-new_name.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_normal_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_normal.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_nocoal_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_no-nuclear_no_coal.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_nox_lim_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_no-nuclear_no_coal_nox_lim.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_renew_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_no-nuclear_no_coal_renewables.csv',usecols=[1,2,4,5,8,22,23,24,25])

oris_nonuc_df.loc[oris_nonuc_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label
oris_normal_df.loc[oris_normal_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label
oris_nocoal_df.loc[oris_nocoal_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label
oris_nox_lim_df.loc[oris_nox_lim_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label
oris_renew_df.loc[oris_renew_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label

###import egrid generation data and modify the index to be ORISCode
egrid_df_raw = pd.read_excel('../raw_data/egrid2016_data.xlsx',sheet_name='GEN16', usecols='D, J, L')
egrid = egrid_df_raw.drop(egrid_df_raw.index[0]).rename(columns={'DOE/EIA ORIS plant or facility code':'ORISCode'})
#change those with no capacity to 1 (so that our capacity factor calculation doesn't fail)
egrid.loc[egrid['Generator nameplate capacity (MW)'] == 0, 'Generator nameplate capacity (MW)'] = 1
#calculate the capacity factor of generation in egrid
egrid['egrid_capafactor'] = egrid['Generator annual net generation (MWh)'] / (8760 * egrid['Generator nameplate capacity (MW)']) 
#update datatype
egrid['Generator nameplate capacity (MW)'] = pd.to_numeric(egrid['Generator nameplate capacity (MW)'])
egrid['Generator annual net generation (MWh)'] = pd.to_numeric(egrid['Generator annual net generation (MWh)'])

###import the generation data from our optimization
gen_normal_df=feather.read_dataframe('../optimization_model/outputs/gen_normal.feather')

###import the generation data from our optimization
gen_nonuc_df=feather.read_dataframe('../optimization_model/outputs/gen_no-nuclear.feather')

###import the generation data from our optimization
gen_nocoal_df=feather.read_dataframe('../optimization_model/outputs/gen_no-nuclear_no-coal.feather')

###import the generation data from our optimization
gen_nox_lim_df=feather.read_dataframe('../optimization_model/outputs/gen_no-nuclear_no-coal.feather')

###import the generation data from our optimization
gen_renew_df=feather.read_dataframe('../optimization_model/outputs/gen_no-nuclear_no-coal_renewables.feather')

### import the plant type from our egrid dataset
planttype_df_raw=pd.read_excel('../raw_data/egrid2016_data.xlsx',sheet_name='PLNT16', usecols='D, W')
planttype_df=planttype_df_raw.drop(egrid_df_raw.index[0]).rename(columns={'DOE/EIA ORIS plant or facility code':'ORISCode', 
                                                                       'Plant primary coal/oil/gas/ other fossil fuel category':'planttype'})



In [11]:
##### change index and convert to xarray #####

#group by ORIS code and make a date and ORIS code multi index
gen_nonuc_df = pd.concat([gen_nonuc_df,oris_nonuc_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_normal_df = pd.concat([gen_normal_df,oris_normal_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_nocoal_df = pd.concat([gen_nocoal_df,oris_nocoal_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_nox_lim_df = pd.concat([gen_nox_lim_df,oris_nox_lim_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_renew_df = pd.concat([gen_renew_df,oris_renew_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()

gen_normal_df = gen_normal_df.stack()
gen_nonuc_df = gen_nonuc_df.stack()
gen_nocoal_df = gen_nocoal_df.stack()
gen_nox_lim_df = gen_nox_lim_df.stack()
gen_renew_df = gen_renew_df.stack()

gen_nonuc_df.index.names = (['ORISCode','date'])
gen_normal_df.index.names = (['ORISCode','date'])
gen_nocoal_df.index.names = (['ORISCode','date'])
gen_nox_lim_df.index.names = (['ORISCode','date'])
gen_renew_df.index.names = (['ORISCode','date'])


egrid = egrid.groupby('ORISCode').sum() #drop capacity factor because that isn't the sum
 
planttype=planttype_df.set_index(['ORISCode'])
planttype_sort=planttype.sort_values(by="ORISCode")

egrid_planttype_df = pd.concat((egrid,planttype_sort),axis=1)

#convert to xarray datasets
oris_nonuc_ds = oris_nonuc_df.to_xarray()
oris_normal_ds = oris_normal_df.to_xarray()
oris_nocoal_ds = oris_nocoal_df.to_xarray()
oris_nox_lim_ds = oris_nox_lim_df.to_xarray()
oris_renew_ds = oris_renew_df.to_xarray()

egrid_planttype_ds = egrid_planttype_df.to_xarray()

gen_normal_ds = gen_normal_df.to_xarray()
gen_nonuc_ds = gen_nonuc_df.to_xarray()
gen_nocoal_ds = gen_nocoal_df.to_xarray()
gen_nox_lim_ds = gen_nox_lim_df.to_xarray()
gen_renew_ds = gen_renew_df.to_xarray()

In [12]:
#### combine our egrid and generation info for no nuclear and normal models ####
nonuc_gmodel_egrid_ds = utils.combine_egrid_generation(oris_nonuc_ds, gen_nonuc_ds, egrid_planttype_ds)
normal_gmodel_egrid_ds = utils.combine_egrid_generation(oris_normal_ds, gen_normal_ds, egrid_planttype_ds)
nocoal_gmodel_egrid_ds = utils.combine_egrid_generation(oris_nocoal_ds, gen_nocoal_ds, egrid_planttype_ds)
noxlim_gmodel_egrid_ds = utils.combine_egrid_generation(oris_nox_lim_ds, gen_nox_lim_ds, egrid_planttype_ds)
renew_gmodel_egrid_ds = utils.combine_egrid_generation(oris_renew_ds, gen_renew_ds, egrid_planttype_ds)

gmodel_egrid_raw_ds = xr.concat([nonuc_gmodel_egrid_ds, normal_gmodel_egrid_ds,nocoal_gmodel_egrid_ds, noxlim_gmodel_egrid_ds,renew_gmodel_egrid_ds], 
                                pd.Index(['nonuc_model','normal_model','nonuc_nocoal_model', 'nox_lim_model', 'renewables_model'], name='model_name'))

In [13]:
#### fix datetime and add dec 31st at the 23rd hour ####

###create a dict for dec 31st 23rd hour (just copy the dec 31 22nd hour data over)
dec23_ds = gmodel_egrid_raw_ds.copy().isel(date = [-1])
#change date to datetime for the 23rd hour
dec23_ds['date'] = [datetime.datetime(2017,12,31,23,0)]

###change all dates to datetime
#make a list of dates for the year
base = datetime.datetime(2017, 1, 1) #base date
date_list = [base + datetime.timedelta(hours=x) for x in range(8759)] #loop through all hours but the final one (we will add it in above)
date_list

gmodel_egrid_raw_ds['date'] = date_list
gmodel_egrid_ds = xr.merge([gmodel_egrid_raw_ds, dec23_ds])

#### Note:

To convert our generation in MWh to get the emissions in kg/sec, we use the following (and making sure our emissions rates are in kg/sec)

$\text{Mwh}/3600sec -> (\text{Mw}/s) * kg/\text{Mw} -> kg/s$


In [14]:
###### add NOx, SO2, CO2, CH4 emissions to gmodel_egrid dataset ######

no_mult = 0.8544304 # NO/NOx as estimated from NEI2011 inventory
no2_mult = 1 - 0.8544304 # NO2/NOx as estimated from NEI2011 inventory

### process our emissions by multiplying generation* emissions factors
gmodel_egrid_ds['model_NO_rate']  = no_mult * gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['model_NO2_rate']  = no2_mult * gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['model_SO2_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLSO2RTA']
gmodel_egrid_ds['model_CO2_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLCO2RTA']
gmodel_egrid_ds['model_CH4_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLCH4RTA']

gmodel_egrid_ds['egrid_annual_NO_rate']  = no_mult * gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['egrid_annual_NO2_rate']  = no2_mult * gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['egrid_annual_SO2_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLSO2RTA']
gmodel_egrid_ds['egrid_annual_CO2_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLCO2RTA']
gmodel_egrid_ds['egrid_annual_CH4_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLCH4RTA']

##### annual generation in original dataset ######
gmodel_egrid_ds['annual_modelgeneration'] = gmodel_egrid_ds['modelgeneration'].groupby('ORISCode').sum(dim = 'date')

In [15]:
####### create new datasets for region and fuel type grouped data, add annual generation to normal ds ######

###list of pollutants
pollutants = ['NO','NO2','SO2','CO2','CH4']

def create_grouped_ds(grouper, model):
    grouped_ds = gmodel_egrid_ds.copy().sel(model_name = model).groupby(grouper).sum()
    grouped_ds = grouped_ds.drop(['model_capafactor','PLCH4RTA','PLCO2RTA','PLNOXRTA','PLSO2RTA','PLN2ORTA'])
    return(grouped_ds)

###create datasets of grouped fuel type and region name, combine
nonuc_fueltype_grouped_ds = create_grouped_ds('fueltype', 'nonuc_model')
normal_fueltype_grouped_ds = create_grouped_ds('fueltype', 'normal_model')
nocoal_fueltype_grouped_ds = create_grouped_ds('fueltype', 'nonuc_nocoal_model')
nox_lim_fueltype_grouped_ds = create_grouped_ds('fueltype', 'nox_lim_model')
renew_fueltype_grouped_ds = create_grouped_ds('fueltype', 'renewables_model')

nonuc_regionname_grouped_ds = create_grouped_ds('regionname', 'nonuc_model')
normal_regionname_grouped_ds = create_grouped_ds('regionname', 'normal_model')
nocoal_regionname_grouped_ds = create_grouped_ds('regionname', 'nonuc_nocoal_model')
nox_lim_regionname_grouped_ds = create_grouped_ds('regionname', 'nox_lim_model')
renew_regionname_grouped_ds = create_grouped_ds('regionname', 'renewables_model')

fueltype_grouped_ds = xr.concat([nonuc_fueltype_grouped_ds, normal_fueltype_grouped_ds,nocoal_fueltype_grouped_ds,nox_lim_fueltype_grouped_ds, renew_fueltype_grouped_ds], 
                                pd.Index(['nonuc_model','normal_model','nonuc_nocoal_model', 'nox_lim_model', 'renewables_model'], name='model_name'))
regionname_grouped_ds = xr.concat([nonuc_regionname_grouped_ds, normal_regionname_grouped_ds, nocoal_regionname_grouped_ds, nox_lim_regionname_grouped_ds, renew_regionname_grouped_ds], 
                                  pd.Index(['nonuc_model','normal_model','nonuc_nocoal_model', 'nox_lim_model', 'renewables_model'], name='model_name'))

In [16]:
#add annual emissions of each pollutant
for pollutant in pollutants:
    regionname_grouped_ds[f'model_annual_{pollutant}_conc'] = (regionname_grouped_ds[f'model_{pollutant}_rate']*3600).sum(dim = 'date')
    fueltype_grouped_ds[f'model_annual_{pollutant}_conc'] = (fueltype_grouped_ds[f'model_{pollutant}_rate']*3600).sum(dim = 'date')
    gmodel_egrid_ds[f'model_annual_{pollutant}_conc'] = (gmodel_egrid_ds[f'model_{pollutant}_rate']*3600).sum(dim = 'date')

    regionname_grouped_ds[f'egrid_annual_{pollutant}_conc'] = (regionname_grouped_ds[f'egrid_annual_{pollutant}_rate']*3600)
    fueltype_grouped_ds[f'egrid_annual_{pollutant}_conc'] = (fueltype_grouped_ds[f'egrid_annual_{pollutant}_rate']*3600)
    gmodel_egrid_ds[f'egrid_annual_{pollutant}_conc'] = (gmodel_egrid_ds[f'egrid_annual_{pollutant}_rate']*3600)

#add ratio of difference in emissions normal-nonuc/ difference in generation normal-nonuc to the nonuc dataset
for pollutant in pollutants:
    fueltype_grouped_ds.sel(model_name = 'nonuc_model')[f'normal-nonuc_{pollutant}-gen_ratio'] = (fueltype_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                       fueltype_grouped_ds.sel(model_name = 'nonuc_model')[f'model_annual_{pollutant}_conc'])/(fueltype_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                             fueltype_grouped_ds.sel(model_name = 'nonuc_model')['annual_modelgeneration'])
    regionname_grouped_ds.sel(model_name = 'nonuc_model')[f'normal-nonuc_{pollutant}-gen_ratio'] = (regionname_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                         regionname_grouped_ds.sel(model_name = 'nonuc_model')[f'model_annual_{pollutant}_conc'])/(regionname_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                                 regionname_grouped_ds.sel(model_name = 'nonuc_model')['annual_modelgeneration'])

    fueltype_grouped_ds.sel(model_name = 'nonuc_nocoal_model')[f'normal-nonuc_nocoal_{pollutant}-gen_ratio'] = (fueltype_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                       fueltype_grouped_ds.sel(model_name = 'nonuc_nocoal_model')[f'model_annual_{pollutant}_conc'])/(fueltype_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                             fueltype_grouped_ds.sel(model_name = 'nonuc_nocoal_model')['annual_modelgeneration'])
    regionname_grouped_ds.sel(model_name = 'nonuc_nocoal_model')[f'normal-nonuc_nocoal_{pollutant}-gen_ratio'] = (regionname_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                         regionname_grouped_ds.sel(model_name = 'nonuc_nocoal_model')[f'model_annual_{pollutant}_conc'])/(regionname_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                                 regionname_grouped_ds.sel(model_name = 'nonuc_nocoal_model')['annual_modelgeneration'])

    fueltype_grouped_ds.sel(model_name = 'nox_lim_model')[f'normal-nonuc_nocoal_{pollutant}-gen_ratio'] = (fueltype_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                       fueltype_grouped_ds.sel(model_name = 'nox_lim_model')[f'model_annual_{pollutant}_conc'])/(fueltype_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                             fueltype_grouped_ds.sel(model_name = 'nox_lim_model')['annual_modelgeneration'])
    regionname_grouped_ds.sel(model_name = 'nox_lim_model')[f'normal-nonuc_nocoal_{pollutant}-gen_ratio'] = (regionname_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                         regionname_grouped_ds.sel(model_name = 'nox_lim_model')[f'model_annual_{pollutant}_conc'])/(regionname_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                                 regionname_grouped_ds.sel(model_name = 'nox_lim_model')['annual_modelgeneration'])

    fueltype_grouped_ds.sel(model_name = 'renewables_model')[f'normal-nonuc_nocoal_{pollutant}-gen_ratio'] = (fueltype_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                       fueltype_grouped_ds.sel(model_name = 'renewables_model')[f'model_annual_{pollutant}_conc'])/(fueltype_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                             fueltype_grouped_ds.sel(model_name = 'renewables_model')['annual_modelgeneration'])
    regionname_grouped_ds.sel(model_name = 'renewables_model')[f'normal-nonuc_nocoal_{pollutant}-gen_ratio'] = (regionname_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                         regionname_grouped_ds.sel(model_name = 'renewables_model')[f'model_annual_{pollutant}_conc'])/(regionname_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                                 regionname_grouped_ds.sel(model_name = 'renewables_model')['annual_modelgeneration'])



In [17]:
#### add attributes ####
for pollutant in pollutants:
    if pollutant == 'NO2' or 'NO':
                gmodel_egrid_ds[f'PLNOXRTA'].attrs['units'] = 'kg/Mwh'
    else:
        gmodel_egrid_ds[f'PL{pollutant}RTA'].attrs['units'] = 'kg/Mwh'
    for idx, ds in enumerate([regionname_grouped_ds, fueltype_grouped_ds, gmodel_egrid_ds]):
        ds[f'model_{pollutant}_rate'].attrs['units'] = 'kg/s'
        ds[f'modelgeneration'].attrs['units'] = 'Mwh'
        for typem in ['egrid','model']:
            ds[f'annual_{typem}generation'].attrs['units'] = 'Mwh'
            ds[f'{typem}_annual_{pollutant}_conc'].attrs['units'] = 'kg'
        ds[f'egrid_annual_{pollutant}_rate'].attrs['units'] = 'kg/s'
        ds.attrs['group'] = ['Region','Fuel Type','All'][idx]
        
        

In [18]:
##### save datasets #####
### Final datasets to save are: gmodel_egrid, fueltype_grouped_ds, regionname_grouped_ds
xr.Dataset.to_zarr(regionname_grouped_ds, './final_data/regionname_grouped_emissions_ds.zarr', mode = 'w') #save the dataset 
xr.Dataset.to_zarr(fueltype_grouped_ds, './final_data/fueltype_grouped_emissions_ds.zarr', mode = 'w') #save the dataset 
xr.Dataset.to_zarr(gmodel_egrid_ds, './final_data/gmodel_egrid_emissions_ds.zarr', mode = 'w') #save the dataset 
### datasets partway to save are:
xr.Dataset.to_zarr(oris_nonuc_ds, './final_data/oris_nonuc_ds.zarr', mode = 'w') #save the dataset 
xr.Dataset.to_zarr(oris_nocoal_ds, './final_data/oris_nonuc_coal_ds.zarr', mode = 'w') #save the dataset 
xr.Dataset.to_zarr(oris_normal_ds, './final_data/oris_normal_ds.zarr', mode = 'w') #save the dataset 
xr.Dataset.to_zarr(egrid_planttype_ds, './final_data/egrid_ds.zarr', mode = 'w') #save the dataset 